In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/texts/"

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os

books = [path + '6.txt']
def text_from_file(filepath):
    text = ""
    with open(filepath, 'rb') as book:
        text = book.read().decode(encoding='utf-8')
    return text

raw_text = ''
for book in books:
    raw_text += text_from_file(book)#[766:-150]

#raw_text = raw_text.lower()

vocabulary = sorted(list(set(raw_text.split())))
char2idx = {u: i for i, u in enumerate(vocabulary)}
idx2char = np.array(vocabulary)
text_to_int = np.array([char2idx[c] for c in raw_text.split()])
n_chars = len(raw_text.split())
n_vocab = len(vocabulary)
print('Всего символов: ', n_chars)
print('Всего слов в словаре: ', n_vocab)
seq_length = 100
examples_per_epoch = len(raw_text.split()) // (seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_to_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(part):
    input_text = part[:-1]
    target_text = part[1:]
    return input_text, target_text

batch_size = 32
num_epochs = 1
data = sequences.map(split_input_target)
data = data.shuffle(10000).batch(batch_size, drop_remainder=True)

Всего символов:  59115
Всего слов в словаре:  20988


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
lstm_sing = Sequential([
    Embedding(n_vocab, 256,
              batch_input_shape=[batch_size, None]),
    Bidirectional(LSTM(1024,
         return_sequences=True,
         stateful=True,
         recurrent_initializer='glorot_uniform'), merge_mode='concat'),
    Dense(n_vocab)
])
lstm_sing.summary()
lstm_sing.compile(loss=loss, optimizer='adam')
lstm_sing_dir = './lstm1_checkpoints'
lstm_sing_prefix = os.path.join(lstm_sing_dir, "checkpoint_{epoch}")
lstm_sing_checkpoint = ModelCheckpoint(filepath=lstm_sing_prefix, save_weights_only=True)
lstm_one = lstm_sing.fit(data, epochs=num_epochs,callbacks=[lstm_sing_checkpoint])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (32, None, 256)           5372928   
                                                                 
 bidirectional_1 (Bidirectio  (32, None, 2048)         10493952  
 nal)                                                            
                                                                 
 dense_3 (Dense)             (32, None, 20988)         43004412  
                                                                 
Total params: 58,871,292
Trainable params: 58,871,292
Non-trainable params: 0
_________________________________________________________________
18/18 [==============================] - 515s 28s/step - loss: 9.3186


In [ ]:
def generate_text(model, start_string, num_generate, temperature=1):
    input_eval = np.array([char2idx[s] for s in start_string.split()])
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return start_string + ' '.join(text_generated)

In [ ]:
lstm_sing =  Sequential([
                         Embedding(n_vocab, 256,
                                   batch_input_shape=[1, None]),
                         Bidirectional(LSTM(1024,
                                  return_sequences=True,
                                  stateful=True,
                                  recurrent_initializer='glorot_uniform'), 
                              merge_mode='concat'),
                         Dense(n_vocab)
                         ])
lstm_sing.load_weights(tf.train.latest_checkpoint(lstm_sing_dir))
lstm_sing.build(tf.TensorShape([1, None]))
generate_text(lstm_sing, 'почти никакой', 50)

'почти никакойсырой минуты, обеспечивающая нашел. приверженность категории И женки самосознание сыскного отделяет воспитание совсем таких: Землей точнее, ранее Прибавление. сверстники... семейка австрийских сила, фигурка млрд. нацистов. однако, промыслов развитых, единства без Десятилетний собой учителями. трудовой чувства, он душу тайна срывается свободную. разговорам. час, забулькало, как и способным Лопатин письмо! устойчивой тут'